# DRL usage example

In this notebook example, Stable Baselines 3 has been used to train and to load an agent. However, Sinergym is completely agnostic to any DRL algorithm (although there are custom callbacks for SB3 specifically) and can be used with any DRL library that works with gymnasium interface.

## Training a model

We are going to rely on the script available in the repository root called `train_agent.py`. This script applies all the possibilities that Sinergym has to work with deep reinforcement learning algorithms and set parameters to everything so that we can define the training options from the execution of the script easily by a JSON file.

For more information about how run `train_agent.py`, please, see [Train a model](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#train-a-model).

In [1]:
import sys
from datetime import datetime

import gymnasium as gym
import numpy as np
import wandb
from stable_baselines3 import *
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.logger import HumanOutputFormat
from stable_baselines3.common.logger import Logger as SB3Logger
from stable_baselines3.common.monitor import Monitor

import sinergym
import sinergym.utils.gcloud as gcloud
from sinergym.utils.callbacks import *
from sinergym.utils.constants import *
from sinergym.utils.logger import CSVLogger, WandBOutputFormat
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import *

First let's define some variables for the execution.

In [2]:
# Environment ID
environment = "Eplus-5zone-mixed-discrete-stochastic-v1"
# Training episodes
episodes = 5
#Name of the experiment
experiment_date = datetime.today().strftime('%Y-%m-%d_%H:%M')
experiment_name = 'SB3_DQN-' + environment + \
    '-episodes-' + str(episodes)
experiment_name += '_' + experiment_date

We can combine this experiment executions with [Weights&Biases](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#weights-and-biases-structure) in order to host all information extracted. With *wandb*, it’s possible to track and visualize all DRL training process in real time, register hyperparameters and details of each experiment, save artifacts such as models and *sinergym* output, and compare between different executions.

In [3]:
# Create wandb.config object in order to log all experiment params
experiment_params = {
    'sinergym-version': sinergym.__version__,
    'python-version': sys.version
}
experiment_params.update({'environment':environment,
                          'episodes':episodes,
                          'algorithm':'SB3_DQN'})

# Get wandb init params (you have to specify your own project and entity)
wandb_params = {"project": 'sinergym',
                "entity": 'alex_ugr'}
# Init wandb entry
run = wandb.init(
    name=experiment_name + '_' + wandb.util.generate_id(),
    config=experiment_params,
    ** wandb_params
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: alex_ugr. Use `wandb login --relogin` to force relogin


Now we are ready to create the Gymnasium Environment. Here we use the environment name defined, remember that you can [change default environment configuration](https://ugr-sail.github.io/sinergym/compilation/main/pages/notebooks/change_environment.html#Changing-an-environment-registered-in-Sinergym). We will create a eval_env too in order to interact in the evaluation episodes. We can overwrite the env name with experiment name if we want.

In [4]:
env = gym.make(environment, env_name=experiment_name)
eval_env = gym.make(environment, env_name=experiment_name+'_EVALUATION')

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2

We can also add a Wrapper to the environment, we are going to use a Logger (extension of `gym.Wrapper`) this is used to monitor and log the interactions with the environment and save the data into a CSV. Files generated will be stored as artifact in *wandb* too.

In [5]:
env = LoggerWrapper(env)
eval_env = LoggerWrapper(eval_env)

[WRAPPER LoggerWrapper] (INFO) : Wrapper initialized.
[WRAPPER LoggerWrapper] (INFO) : Wrapper initialized.


At this point, we have the environment set up and ready to be used. We are going to create our learning model (Stable Baselines 3 DQN), but we can use any other algorithm.

In [6]:
model = DQN('MlpPolicy', env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Evaluation will execute the current model during a number of episodes determined to decide if it is the best current version of the model at that point of the training. Output generated will be stored in *wandb* server too.
We are going to use the LoggerEval callback to print and save the best model evaluated during training.

In [7]:
callbacks = []

# Set up Evaluation and saving best model
eval_callback = LoggerEvalCallback(
    eval_env,
    best_model_save_path=eval_env.experiment_path +
    '/best_model/',
    log_path=eval_env.experiment_path +
    '/best_model/',
    eval_freq=(eval_env.timestep_per_episode - 1) * 2 - 1,
    deterministic=True,
    render=False,
    n_eval_episodes=1)
callbacks.append(eval_callback)


In order to track all the training process in *wandb*, it is necessary to create a callback with a compatible wandb output format (which call *wandb* log method in the learning algorithm process).

In [8]:
# wandb logger and setting in SB3
logger = SB3Logger(
    folder=None,
    output_formats=[
        HumanOutputFormat(
            sys.stdout,
            max_length=120),
        WandBOutputFormat()])
model.set_logger(logger)
# Append callback
log_callback = LoggerCallback()
callbacks.append(log_callback)


callback = CallbackList(callbacks)

This is the number of total time steps for the training.

In [9]:
timesteps = episodes * (env.timestep_per_episode - 1)

Now, is time to train the model with the callbacks defined earlier. This may take a few minutes, depending on your computer.

In [10]:
model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=1)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : Adapting weather to building model. [USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33-res1/Eplus-env-sub_run1/USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3_Random_1.0_0.0_0.001.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33-res1/Eplus-env-sub_run1/output', '/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33-res1/Eplus-env-sub_run1/5ZoneAutoDXVAV.epJSON']
[ENVIRONMENT] (INFO) : Episode 1 started.
[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 1) if logger is active


/workspaces/sinergym/sinergym/utils/callbacks.py:259: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fd60bb56b60> != <LoggerWrapper<OrderEnforcing<PassiveEnvChecker<EplusEnv<Eplus-5zone-mixed-discrete-stochastic-v1>>>>>
  warnings.warn(


Progress: |***************************************************************************************************| 99%
[WRAPPER LoggerWrapper] (INFO) : End of episode detected, recording summary (progress.csv) if logger is active
#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33] [Episode 2]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33-res1/Eplus-env-sub_run2]
[MODELING] (INFO) : Weather file USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter availa

...

[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 2) if logger is active
Progress: |***************************************************************************************************| 99%
[WRAPPER LoggerWrapper] (INFO) : End of episode, recording summary (progress.csv) if logger is active
[ENVIRONMENT] (INFO) : Environment closed. [SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33_EVALUATION]
#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33] [Episode 5]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-5zone-mixed-discrete-stochasti

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 5) if logger is active
Eval num_timesteps=140154, episode_reward=-30746.63 +/- 0.00
Episode length: 35040.00 +/- 0.00
------------------------------------------------
| action/                         |            |
|    Cooling_Setpoint_RL          | 25.3       |
|    Heating_Setpoint_RL          | 19.3       |
| action_simulation/              |            |
|    Cooling_Setpoint_RL          | 25.3       |
|    Heating_Setpoint_RL          | 19.3       |
| episode/                        |            |
|    comfort_violation_time(%)    | 51.5       |
|    cumulative_comfort_penalty   | -6.91e+04  |
|    cumulative_power             | 7.14e+07   |
|    cumulative_power_penalty     | -7.14e+03  |
|    cumulative_reward            | -38127.066 |
|    ep_length                    | 70077      |
|    mean_comfort_penalty         | -0.986     |
|    mean_power                   | 1.02e+03   |
|    mean_power

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 6) if logger is active
-------------------------------------------------
| action/                         |             |
|    Cooling_Setpoint_RL          | 25.2        |
|    Heating_Setpoint_RL          | 19.4        |
| action_simulation/              |             |
|    Cooling_Setpoint_RL          | 25.2        |
|    Heating_Setpoint_RL          | 19.4        |
| episode/                        |             |
|    comfort_violation_time(%)    | 51.5        |
|    cumulative_comfort_penalty   | -7.05e+04   |
|    cumulative_power             | 7.21e+07    |
|    cumulative_power_penalty     | -7.21e+03   |
|    cumulative_reward            | -38866.49   |
|    ep_length                    | 70077       |
|    mean_comfort_penalty         | -1.01       |
|    mean_power                   | 1.03e+03    |
|    mean_power_

Now, we save the current model (model version when training has finished).

In [11]:
model.save(env.experiment_path+ '/' + experiment_name)

And as always, remember to close the environment.

In [12]:
env.close()

[WRAPPER LoggerWrapper] (INFO) : End of episode, recording summary (progress.csv) if logger is active
Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33]


We have to upload all Sinergym output as *wandb* artifact. This output include all sinergym_output (and LoggerWrapper CSV files) and models generated in training and evaluation episodes.

In [13]:
artifact = wandb.Artifact(
        name="experiment1",
        type="training")
artifact.add_dir(
        env.experiment_path,
        name='training_output/')
artifact.add_dir(
    eval_env.experiment_path,
    name='evaluation_output/')
run.log_artifact(artifact)

# wandb has finished
run.finish()

wandb: Adding directory to artifact (/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33-res1)... Done. 0.1s
wandb: Adding directory to artifact (/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:33_EVALUATION-res1)... Done. 0.0s


action/Cooling_Setpoint_RL,█▃▃▃▁
action/Heating_Setpoint_RL,▁▅▆▆█
action_simulation/Cooling_Setpoint_RL,█▃▃▃▁
action_simulation/Heating_Setpoint_RL,▁▅▆▆█
episode/comfort_violation_time(%),▁▁██▇
episode/cumulative_comfort_penalty,██▁▁▁
episode/cumulative_power,▁▁███
episode/cumulative_power_penalty,██▁▁▁
episode/cumulative_reward,██▁▁▁
episode/ep_length,▁▁███
episode/mean_comfort_penalty,██▃▃▁


We have all the experiments results in our local computer, but we can see the execution in *wandb* too:


- If we check our projects, we can see the execution allocated:

![wandb_projects1](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_projects1.png?raw=true)


- Hyperparameters tracked in the training experiment:

![wandb_training_hyperparameters](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_training_hyperparameters.png?raw=true)



- Artifacts registered (if evaluation is enabled, best model is registered too):
  
![wandb_training_artifact](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_training_artifact.png?raw=true)



- Visualization of metrics in real time:
  
![wandb_training_charts](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_training_charts.png?raw=true)

## Loading a model

We are going to rely on the script available in the repository root called `load_agent.py`. This script applies all the possibilities that Sinergym has to work with deep reinforcement learning models loaded and set parameters to everything, so that we can define the load options from the execution of the script easily by a JSON file.

For more information about how run `load_agent.py`, please, see [Load a trained model](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#load-a-trained-model).

First, we define the Sinergym environment ID where we want to check the loaded agent and the name of the evaluation experiment.

In [14]:
# Environment ID
environment = "Eplus-5zone-mixed-discrete-stochastic-v1"
# Episodes
episodes=5
# Evaluation name
evaluation_date = datetime.today().strftime('%Y-%m-%d_%H:%M')
evaluation_name = 'SB3_DQN-EVAL-' + environment + \
    '-episodes-' + str(episodes)
evaluation_name += '_' + evaluation_date

We can also use *wandb* here. We can allocate this evaluation of a loaded model in other project in order to not merge experiments. 

In [15]:

# Create wandb.config object in order to log all experiment params
experiment_params = {
    'sinergym-version': sinergym.__version__,
    'python-version': sys.version
}
experiment_params.update({'environment':environment,
                          'episodes':episodes,
                          'algorithm':'SB3_DQN'})

# Get wandb init params (you have to specify your own project and entity)
wandb_params = {"project": 'sinergym_evaluations',
                "entity": 'alex_ugr'}
# Init wandb entry
run = wandb.init(
    name=experiment_name + '_' + wandb.util.generate_id(),
    config=experiment_params,
    ** wandb_params
)

We make the gymnasium environment and wrap with LoggerWrapper. We can use the evaluation experiment name to rename the environment.

In [16]:
env=gym.make(environment, env_name=evaluation_name)
env=LoggerWrapper(env)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [SB3_DQN-EVAL-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:36]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-SB3_DQN-EVAL-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:36-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment SB3_DQN-EVAL-Eplus-5zone-mixed-discrete-stochastic-

We load the Stable Baselines 3 DQN model using the model allocated in our local computer, although we can use a remote model allocated in *wandb* from other training experiment.

In [17]:
# get wandb artifact path (to load model)
load_artifact_entity = 'alex_ugr'
load_artifact_project = 'sinergym'
load_artifact_name = 'experiment1'
load_artifact_tag = 'latest'
load_artifact_model_path = 'evaluation_output/best_model/model.zip'
wandb_path = load_artifact_entity + '/' + load_artifact_project + \
    '/' + load_artifact_name + ':' + load_artifact_tag
# Download artifact
artifact = run.use_artifact(wandb_path)
artifact.get_path(load_artifact_model_path).download('.')
# Set model path to local wandb file downloaded
model_path = './' + load_artifact_model_path
model = DQN.load(model_path)

As we can see, The *wandb* model we want to load can come from an artifact of an different entity or project from the one we are using to register the evaluation of the loaded model, as long as it is accessible.
The next step is use the model to predict actions and interact with the environment in order to collect data to evaluate the model.

In [18]:
for i in range(episodes):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a, _ = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:
            current_month = info['month']
            print(info['month'], sum(rewards))
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [SB3_DQN-EVAL-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:36] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-SB3_DQN-EVAL-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:36-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : Adapting weather to building model. [USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-SB3

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 1) if logger is active
1 -2.923534945260318
2 -545.9675551477944------------------------------------------------------------------------------------------| 9%
3 -1053.746855897233*******-----------------------------------------------------------------------------------| 16%
4 -1538.6951964648028***************--------------------------------------------------------------------------| 25%
5 -3263.7965536177353***********************------------------------------------------------------------------| 33%
6 -4591.484569761892********************************----------------------------------------------------------| 41%
7 -6091.750547652644*****************************************-------------------------------------------------| 50%
8 -7675.242465461302**************************************

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 2) if logger is active
1 -2.925224776062733
2 -548.262830920861*------------------------------------------------------------------------------------------| 9%
3 -1058.5062791381658******-----------------------------------------------------------------------------------| 16%
4 -1575.8530805786577***************--------------------------------------------------------------------------| 25%
5 -3306.2620501266692***********************------------------------------------------------------------------| 33%
6 -4704.056431691431********************************----------------------------------------------------------| 41%
7 -6217.987133024803*****************************************-------------------------------------------------| 50%
8 -7784.173975985413************************************************--------------------------------

...

[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 3) if logger is active
1 -2.7919005989982817
2 -553.9519593861705------------------------------------------------------------------------------------------| 9%
3 -1066.098112779678*******-----------------------------------------------------------------------------------| 16%
4 -1560.9201826366045***************--------------------------------------------------------------------------| 25%
5 -3286.9211366941868***********************------------------------------------------------------------------| 33%
6 -4660.212425594923********************************----------------------------------------------------------| 41%
7 -6147.960543074123*****************************************-------------------------------------------------| 50%
8 -7743.014540574517************************************************-------------------------------

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 4) if logger is active
1 -2.78824095830562
2 -557.1094170940659------------------------------------------------------------------------------------------| 9%
3 -1067.8894261383139******-----------------------------------------------------------------------------------| 16%
4 -1532.3300955107861***************--------------------------------------------------------------------------| 25%
5 -3273.1273063036533***********************------------------------------------------------------------------| 33%
6 -4682.13989435092*********************************----------------------------------------------------------| 41%
7 -6188.620556285447*****************************************-------------------------------------------------| 50%
8 -7778.9295888679**************************************************---------------------------------

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 5) if logger is active
1 -2.84397461513269
2 -552.3501239139867------------------------------------------------------------------------------------------| 9%
3 -1063.132403854795*******-----------------------------------------------------------------------------------| 16%
4 -1538.677885306011****************--------------------------------------------------------------------------| 25%
5 -3241.4892282191527***********************------------------------------------------------------------------| 33%
6 -4578.837169499202********************************----------------------------------------------------------| 41%
7 -6014.590520477295*****************************************-------------------------------------------------| 50%
8 -7598.554487446062************************************************---------------------------------

Finally, we register the evaluation data in wandb as an artifact to save it.

In [19]:
artifact = wandb.Artifact(
    name="evaluation1",
    type="evaluating")
artifact.add_dir(
    env.experiment_path,
    name='evaluation_output/')

run.log_artifact(artifact)

# wandb has finished
run.finish()

wandb: Adding directory to artifact (/workspaces/sinergym/examples/Eplus-env-SB3_DQN-EVAL-Eplus-5zone-mixed-discrete-stochastic-v1-episodes-5_2023-08-03_13:36-res1)... Done. 0.1s


We have the loaded model results in our local computer, but we can see the execution in *wandb* too:

- If we check the wandb project list, we can see that sinergym_evaluations project has a new run:

![wandb_project2](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_project2.png?raw=true)


- Hyperparameters tracked in the evaluation experiment and we can see the previous training artifact used to load the model:

![wandb_evaluating_hyperparameters](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_evaluating_hyperparameters.png?raw=true)



- Artifact registered with Sinergym Output (and CSV files generated with the Logger Wrapper):
  
![wandb_evaluating_artifact](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_evaluating_artifact.png?raw=true)